In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split, StratifiedKFold
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import numpy as np
import tqdm.notebook as tq
import itertools
import pickle

In [2]:
train = pd.read_csv("Train.csv")
#train = train[train["Label"].isin(["POLITICS", "SOCIAL", "RELIGION", "LAW/ORDER", "SOCIAL ISSUES", "HEALTH", "ECONOMY", "FARMING"])]
train_data = train["Text"]
train_labels = train["Label"]

labels = train_labels.unique()
print(train_labels.value_counts() / len(train_labels))

for i in range(len(labels)):
    train_labels[train_labels == labels[i]] = i

POLITICS                0.194290
SOCIAL                  0.105850
RELIGION                0.102368
LAW/ORDER               0.094708
SOCIAL ISSUES           0.093315
HEALTH                  0.088440
ECONOMY                 0.059889
FARMING                 0.054318
SPORTS                  0.034123
EDUCATION               0.029944
RELATIONSHIPS           0.027159
WILDLIFE/ENVIRONMENT    0.025070
OPINION/ESSAY           0.018106
LOCALCHIEFS             0.017409
CULTURE                 0.016017
WITCHCRAFT              0.011142
MUSIC                   0.010446
TRANSPORT               0.007660
ARTS AND CRAFTS         0.004875
FLOODING                0.004875
Name: Label, dtype: float64


In [3]:
train_sentences, test_sentences, y_train, y_test = train_test_split(train_data, train_labels, test_size = .1, stratify = train_labels)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_sentences)
x_train = tokenizer.texts_to_sequences(train_sentences)
x_test = tokenizer.texts_to_sequences(test_sentences)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
print(train_sentences[0])
print(x_train[0])
maxLen = max([len(x_train[i]) for i in range(len(x_train))])
print(maxLen)
x_train = pad_sequences(x_train, padding = 'post', maxlen=maxLen)
x_test = pad_sequences(x_test, padding = 'post', maxlen=maxLen)
x_train = np.array(x_train, dtype=np.float)
x_test = np.array(x_test, dtype=np.float)
y_train = np.array(y_train, dtype=np.float)
y_test = np.array(y_test, dtype=np.float)

47466
 Mwangonde: Khansala wachinyamata Akamati achinyamata ndi atsogoleri a mawa, ambiri amaganiza kuti izi ndi nkhambakamwa chabe. Koma achinyamata ena, monga Lusubilo Mwangonde, akukwaniritsa akupherezetsa mawuwa osati pongolota kuti adzakhala, koma kutsogolera kumene chifukwa nthawi yawo yakwana. DAILES BANDA adacheza ndi Mwangonde, khansala wachinyama, yemwe akuimira Jumbo Ward mumzinda wa Mzuzu, motere:  Chisale watuluka nkumangidwanso  Sipakala waimitsa Nyumba ya Malamulo  Pa Wenela pasintha zedi Ali ndi masomphenya: Mwangonde Tikudziweni  Ndine Lusubilo Mwangonde, ndili ndi zaka 27 zakubadwa. Ndinabadwa mbanja la ana asanu ndipo ndine wachinayi kubadwa. Ndimachokera mmudzi mwa Mwamalopa, kwa Paramount Chief Kyungu mboma la Karonga. Sindili pabanja pakadalipano.
 Mbiri ya maphunziro anu ndi yotani? Maphunziro anga a pulaimale ndidachitira kusukula yapulaiveti ya Viphya mumzinda wa Mzuzu ndipo asekondale ndidachitira pa Phwezi Boys mboma la Rumphi. Ndili ndi diploma ya Accounting

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWa

In [4]:
embedding_dim = 100

model = keras.models.Sequential()
model.add(keras.layers.Embedding(input_dim = vocab_size, output_dim = embedding_dim, input_length = maxLen))
model.add(keras.layers.Conv1D(128, 5, activation='relu'))
model.add(keras.layers.GlobalMaxPool1D())
model.add(keras.layers.Dense(20, activation = 'relu'))
model.add(keras.layers.Dense(20, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 909, 100)          4746600   
                                                                 
 conv1d (Conv1D)             (None, 905, 128)          64128     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 20)                2580      
                                                                 
 dense_1 (Dense)             (None, 20)                420       
                                                                 
Total params: 4,813,728
Trainable params: 4,813,728
Non-trainable params: 0
______________________________________________

In [ ]:
model.fit(x_train, y_train, epochs = 30, validation_data = (x_test, y_test))

In [6]:
model.evaluate(x_test, y_test, verbose=False)[1]

0.5277777910232544

In [ ]:
vectorizer = TfidfVectorizer()
x_train_vec = vectorizer.fit_transform(train_sentences)
x_test_vec = vectorizer.transform(test_sentences)
x_train_vec.shape

(1292, 47229)

In [3]:
def createVecModel(input_size, denseLayer, activation, dropout):
    vecModel = keras.models.Sequential()
    vecModel.add(keras.layers.InputLayer(input_size))
    vecModel.add(keras.layers.Dense(denseLayer, activation=activation))
    vecModel.add(keras.layers.Dropout(dropout))
    vecModel.add(keras.layers.Dense(20, activation='softmax'))
    vecModel.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return vecModel

In [4]:
params = [[500, 600, 700, 800, 900, 1000],
          ['elu', 'relu', 'selu', 'tanh'],
          [.6, .8, .9]
         ]
params = itertools.product(*params)
params = list(params)
len(params)

72

In [5]:
skf = StratifiedKFold(n_splits=4)
vectorizer = TfidfVectorizer()
best_acc = 0
best_params = None
for combo in tq.tqdm(params):
    denseLayer, activation, dropout = combo
    acc = 0
    for trainidx, testidx in skf.split(train_data, LabelEncoder().fit_transform(train_labels)):
        x_train, y_train = train_data[trainidx], train_labels[trainidx]
        x_test, y_test = train_data[testidx], train_labels[testidx]
        
        x_train_vec = vectorizer.fit_transform(x_train)
        x_test_vec = vectorizer.transform(x_test)
        
        x_train_vec = x_train_vec.toarray()
        x_test_vec = x_test_vec.toarray()
        y_train = np.array(y_train, dtype='float')
        y_test = np.array(y_test, dtype='float')
        
        vecModel = createVecModel(x_train_vec.shape[1], denseLayer, activation, dropout)
        vecModel.fit(x_train_vec, y_train, epochs = 12, verbose = False)
        acc += vecModel.evaluate(x_test_vec, y_test, verbose = False)[1]

        del vecModel
    acc /= 4
    
    if acc > best_acc:
        best_acc = acc
        best_params = (denseLayer, activation, dropout)
        print("Best Params: ", best_params)
        print("Best Accuracy: ", best_acc)
    


print(best_params, best_acc)
with open("best_params_acc.txt", "wb") as file:
    pickle.dump((best_params, best_acc), file)

  0%|          | 0/72 [00:00<?, ?it/s]

Best Params:  (500, 'elu', 0.6)
Best Accuracy:  0.6420612931251526
Best Params:  (500, 'elu', 0.8)
Best Accuracy:  0.6434540450572968
Best Params:  (500, 'selu', 0.9)
Best Accuracy:  0.6448467969894409
Best Params:  (600, 'selu', 0.9)
Best Accuracy:  0.6476323157548904
(600, 'selu', 0.9) 0.6476323157548904
